In [1]:
#using pytorch

!pip3 install torch torchvision torchaudio

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.4 MB 656.4 kB/s eta 0:00:04
   - -------------------------------------- 0.1/2.4 MB 819.2 kB/s eta 0:00:03
   --- ------------------------------------ 0.2/2.4 MB 1.3 MB/s eta 0:00:02
   ------ --------------------------------- 0.4/2.4 MB 1.7 MB/s eta 0:00:02
   -------- ------------------------------- 0.5/2.4 MB 2.0 MB/s eta 0:00:01
   ----------- ---------------------------- 0.7/2.4 MB 2.2 MB/s eta 0:00:01
   ---------------- ----------------------- 1.0/2.4 MB 2.7 MB/s eta 0:00:01
   ------------------- -------------------- 1.2/2.4 MB 2.9 MB/s eta 0:00:01
   ----------------------- ---------------- 1.4/2.4 MB 3.1 MB/s eta 0:00:01
   -------------------------- ------------- 1.6/2.4 MB 3.1 MB/s eta 0:00:01
   ------------------------------ --------- 1.8/2.4 MB 3.2 MB/s eta 0:00:01
   -------------------

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from torch.utils.data import DataLoader, Dataset
import torch.nn as nn

In [3]:
class plot_error_surfaces(object):
    def __init__(self, w_range, b_range, X, Y, n_samples = 30, go = True):
        W = np.linspace(-w_range, w_range, n_samples)
        B  = np.linspace(-b_range, b_range, n_samples)
        w, b = np.meshgrid(W, B)
        Z = np.zeros((30, 30))
        count1 = 0
        
        self.y = Y.numpy()
        self.x = X.numpy()
        
        for w1, b1 in zip(w, b):
            count2 = 0
            for w2, b2 in zip(w1, b1):
                yhat = 1/(1 + np.exp(-1*(w2*self.x + b2)))
                Z[count1, count2] = -1*np.mean(self.y*np.log(yhat+1e-16) + (1-self.y)*np.log(1-yhat+1e-16))
                count2 += 1
        self.Z = Z
        self.w = w
        self.b = b
        self.W = []
        self.B = []
        self.LOSS = []
        self.n = 0
        if go == True:
            plt.figure()
            plt.figure(figsize=(7.5, 5))
            plt.axes(projection='3d').plot_surface(self.w, self.b, self.Z, rstride=1, cstride=1, cmap='viridis', edgecolor='none')
            plt.title('Loss Surface')
            plt.xlabel('w')
            plt.ylabel('b')
            plt.show()
            plt.figure()
            plt.title('Loss Surface Contour')
            plt.xlabel('w')
            plt.ylabel('b')
            plt.contour(self.w, self.b, self.Z)
            plt.show()
            
    def set_para_loss(self, model, loss):
        self.n = self.n + 1
        self.LOSS.append(loss)
        self.W.append(list(model.parameters())[0].item())
        self.B.append(list(model.parameters())[1].item())
        
    def final_plot(self):
        ax = plt.axes(projection='3d')
        ax.plot_wireframe(self.w, self.b, self.Z)
        ax.scatter(self.W, self.B, self.LOSS, c='r', marker='x', s=200, alpha=1)
        plt.figure()
        plt.contour(self.w, self.b, self.Z)
        plt.scatter(self.W, self.B, c='r', marker='x')
        plt.xlabel('w')
        plt.ylabel('b')
        plt.show()
    
    def plot_ps(self):
        plt.subplot(121)
        plt.ylim
        plt.plot(self.x[self.y==0], self.y[self.y==0], 'ro', label="training points")
        plt.plot(self.x[self.y==1], self.y[self.y==1]-1, 'o', label="training points")
        plt.plot(self.x, self.W[-1] * self.x + self.B[-1], label="estimated line")
        plt.xlabel('x')
        plt.ylabel('y')
        plt.ylim((-0.1, 2))
        plt.title('Data Space Iteration: ' + str(self.n))
        plt.show()
        plt.subplot(122)
        plt.contour(self.w, self.b, self.Z)
        plt.scatter(self.W, self.B, c='r', marker='x')
        plt.title('Loss Surface Contour Iteration' + str(self.n))
        plt.xlabel('w')
        plt.ylabel('b')   
        
    def PlotStuff(X, Y, model, epoch, leg=True):
        plt.plot(X.numpy(), model(X).detach().numpy(), label='epoch ' + str(epoch))
        plt.plot(X.numpy(), Y.numpy(), 'r')
        if leg == True:
            plt.legend()
        else:
            pass

In [4]:
#set a random seed
torch.manual_seed(0)

In [ ]:
# Create the custom Data class which inherits Dataset
class Data(Dataset):
    
    # Constructor
    def __init__(self):
        # Create X values from -1 to 1 with step .1
        self.x = torch.arange(-1, 1, 0.1).view(-1, 1)
        # Create Y values all set to 0
        self.y = torch.zeros(self.x.shape[0], 1)
        # Set the X values above 0.2 to 1
        self.y[self.x[:, 0] > 0.2] = 1
        # Set the .len attribute because we need to override the __len__ method
        self.len = self.x.shape[0]
    
    # Getter that returns the data at the given index
    def __getitem__(self, index):      
        return self.x[index], self.y[index]
    
    # Get length of the dataset
    def __len__(self):
        return self.len

In [5]:
# import cv2

# def list_camera_indices():
#     # Iterate over a range of possible camera indices
#     for i in range(10):
#         # Initialize a VideoCapture object with the current index
#         cap = cv2.VideoCapture(i)
        
#         # Check if the camera is opened successfully
#         if cap.isOpened():
#             print(f"Camera index {i} is available")
#             # Release the VideoCapture object
#             cap.release()
#         else:
#             print(f"Camera index {i} is not available")

# if __name__ == "__main__":
#     list_camera_indices()


Camera index 0 is available
Camera index 1 is not available
Camera index 2 is not available
Camera index 3 is not available
Camera index 4 is not available
Camera index 5 is not available
Camera index 6 is not available
Camera index 7 is not available
Camera index 8 is not available
Camera index 9 is not available
